In [ ]:
import pandas as pd
import numpy as np

In [ ]:
filepath = "C:\\Users\\hlmq\\OneDrive - Chevron\\Data\\WeatherProject\\"

#df_2018 = pd.read_csv(str(filepath)+'2018.csv')
#df_2019 = pd.read_csv(str(filepath)+'2019.csv')
#df_2020 = pd.read_csv(str(filepath)+'2020.csv')
#df_2021 = pd.read_csv(str(filepath)+'2021.csv')
allDates = pd.read_csv(str(filepath)+'allDates.csv')

In [ ]:
df = allDates

# Features

In [ ]:
# Construct binary column for precipitation

df.loc[df['Precipitation'] < .1, 'Precipitation_Bin'] = 0
df.loc[df['Precipitation'] > .1, 'Precipitation_Bin'] = 1

# Export Dataset

In [ ]:
# Modify filepath and file name

filepath = "C:\\Users\\hlmq\\OneDrive - Chevron\\Data\\WeatherProject\\"
filename = ""

df.to_csv(str(filepath)+str(filename))